In [ ]:
import mlflow
import pandas as pd
import mlflow.sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import numpy as np

from dotenv import load_dotenv
import os


load_dotenv()

DAGSHUB_URL = os.getenv("DAGSHUB_URL")
GITHUB_OWNER = os.getenv("GITHUB_OWNER")
PROJECT_NAME = os.getenv("PROJECT_NAME")

https://dagshub.com/keshav-github-123/Text-Classification-MLOPS.mlflow keshav-github-123 Text-Classification-MLOPS


In [5]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet') 

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\KESHAV\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\KESHAV\AppData\Roaming\nltk_data...


True

In [6]:
df = pd.read_csv('IMDB.csv')
df = df.sample(500)
df.to_csv('data.csv', index=False)
df.head()

,review,sentiment
199,Your ability to enjoy The Ashes of Time may de...,negative
353,I thought this had the right blend of characte...,positive
313,A man named Jerry comes into possession of an ...,positive
868,The movie is good and I think Tiffany Amber is...,positive
69,Before Stan Laurel became the smaller half of ...,negative


In [7]:
# data preprocessing

# Define text preprocessing functions
def lemmatization(text):
    """Lemmatize the text."""
    lemmatizer = WordNetLemmatizer()
    text = text.split()
    text = [lemmatizer.lemmatize(word) for word in text]
    return " ".join(text)

def remove_stop_words(text):
    """Remove stop words from the text."""
    stop_words = set(stopwords.words("english"))
    text = [word for word in str(text).split() if word not in stop_words]
    return " ".join(text)

def removing_numbers(text):
    """Remove numbers from the text."""
    text = ''.join([char for char in text if not char.isdigit()])
    return text

def lower_case(text):
    """Convert text to lower case."""
    text = text.split()
    text = [word.lower() for word in text]
    return " ".join(text)

def removing_punctuations(text):
    """Remove punctuations from the text."""
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = text.replace('؛', "")
    text = re.sub('\s+', ' ', text).strip()
    return text

def removing_urls(text):
    """Remove URLs from the text."""
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def normalize_text(df):
    """Normalize the text data."""
    try:
        df['review'] = df['review'].apply(lower_case)
        df['review'] = df['review'].apply(remove_stop_words)
        df['review'] = df['review'].apply(removing_numbers)
        df['review'] = df['review'].apply(removing_punctuations)
        df['review'] = df['review'].apply(removing_urls)
        df['review'] = df['review'].apply(lemmatization)
        return df
    except Exception as e:
        print(f'Error during text normalization: {e}')
        raise

<>:32: SyntaxWarning: invalid escape sequence '\s'
<>:32: SyntaxWarning: invalid escape sequence '\s'
C:\Users\KESHAV\AppData\Local\Temp\ipykernel_12216\3798096103.py:32: SyntaxWarning: invalid escape sequence '\s'
  text = re.sub('\s+', ' ', text).strip()


In [8]:
df = normalize_text(df)
df.head()

,review,sentiment
199,ability enjoy ash time may depend expectation ...,negative
353,thought right blend character plot futuristic ...,positive
313,man named jerry come possession ancient aztec ...,positive
868,movie good think tiffany amber beautiful liked...,positive
69,stan laurel became smaller half all time great...,negative


In [9]:
df['sentiment'].value_counts()

sentiment
negative    250
positive    250
Name: count, dtype: int64

In [10]:
x = df['sentiment'].isin(['positive','negative'])
df = df[x]

In [11]:
df['sentiment'] = df['sentiment'].map({'positive':1, 'negative':0})
df.head()

,review,sentiment
199,ability enjoy ash time may depend expectation ...,0
353,thought right blend character plot futuristic ...,1
313,man named jerry come possession ancient aztec ...,1
868,movie good think tiffany amber beautiful liked...,1
69,stan laurel became smaller half all time great...,0


In [12]:
df.isnull().sum()

review       0
sentiment    0
dtype: int64

In [18]:
vectorizer = CountVectorizer(max_features=50)
X = vectorizer.fit_transform(df['review'])
y = df['sentiment']

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
import dagshub

mlflow.set_tracking_uri(DAGSHUB_URL)
dagshub.init(repo_owner=GITHUB_OWNER, repo_name=PROJECT_NAME, mlflow=True)

# mlflow.set_experiment("Logistic Regression Baseline")
mlflow.set_experiment("Logistic Regression Baseline")


2025-10-31 12:45:15,489 - INFO - HTTP Request: GET https://dagshub.com/api/v1/repos/keshav-github-123/Text-Classification-MLOPS "HTTP/1.1 200 OK"


Initialized MLflow to track repo "keshav-github-123/Text-Classification-MLOPS"

2025-10-31 12:45:15,494 - INFO - Initialized MLflow to track repo "keshav-github-123/Text-Classification-MLOPS"


Repository keshav-github-123/Text-Classification-MLOPS initialized!

2025-10-31 12:45:15,495 - INFO - Repository keshav-github-123/Text-Classification-MLOPS initialized!


<Experiment: artifact_location='mlflow-artifacts:/a9d6560f20ed4212956fcb5eb4b47fae', creation_time=1761894419257, experiment_id='0', last_update_time=1761894419257, lifecycle_stage='active', name='Logistic Regression Baseline', tags={}>

In [21]:
import mlflow
import logging
import os
import time
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Configure logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

logging.info("Starting MLflow run...")

with mlflow.start_run():
    start_time = time.time()
    
    try:
        logging.info("Logging preprocessing parameters...")
        mlflow.log_param("vectorizer", "Bag of Words")
        mlflow.log_param("num_features", 50)
        mlflow.log_param("test_size", 0.2)

        logging.info("Initializing Logistic Regression model...")
        model = LogisticRegression(max_iter=1000)  # Increase max_iter to prevent non-convergence issues

        logging.info("Fitting the model...")
        model.fit(X_train, y_train)
        logging.info("Model training complete.")

        logging.info("Logging model parameters...")
        mlflow.log_param("model", "Logistic Regression")

        logging.info("Making predictions...")
        y_pred = model.predict(X_test)

        logging.info("Calculating evaluation metrics...")
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)

        logging.info("Logging evaluation metrics...")
        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric("precision", precision)
        mlflow.log_metric("recall", recall)
        mlflow.log_metric("f1_score", f1)

        logging.info("Saving and logging the model...")
        # mlflow.sklearn.log_model(model, "model")
        mlflow.sklearn.save_model(model, "saved_model")
        mlflow.log_artifact("saved_model")


        # Log execution time
        end_time = time.time()
        logging.info(f"Model training and logging completed in {end_time - start_time:.2f} seconds.")

        # Save and log the notebook
        # notebook_path = "exp1_baseline_model.ipynb"
        # logging.info("Executing Jupyter Notebook. This may take a while...")
        # os.system(f"jupyter nbconvert --to notebook --execute --inplace {notebook_path}")
        # mlflow.log_artifact(notebook_path)

        # logging.info("Notebook execution and logging complete.")

        # Print the results for verification
        logging.info(f"Accuracy: {accuracy}")
        logging.info(f"Precision: {precision}")
        logging.info(f"Recall: {recall}")
        logging.info(f"F1 Score: {f1}")

    except Exception as e:
        logging.error(f"An error occurred: {e}", exc_info=True)


2025-10-31 12:45:24,691 - INFO - Starting MLflow run...
2025-10-31 12:45:25,526 - INFO - Logging preprocessing parameters...
2025-10-31 12:45:26,954 - INFO - Initializing Logistic Regression model...
2025-10-31 12:45:26,954 - INFO - Fitting the model...
2025-10-31 12:45:26,970 - INFO - Model training complete.
2025-10-31 12:45:26,970 - INFO - Logging model parameters...
2025-10-31 12:45:27,365 - INFO - Making predictions...
2025-10-31 12:45:27,367 - INFO - Calculating evaluation metrics...
2025-10-31 12:45:27,377 - INFO - Logging evaluation metrics...
2025-10-31 12:45:29,005 - INFO - Saving and logging the model...
2025-10-31 12:45:29,006 - ERROR - An error occurred: Path 'saved_model' already exists and is not empty
Traceback (most recent call last):
  File "C:\Users\KESHAV\AppData\Local\Temp\ipykernel_12216\2921203159.py", line 49, in <module>
    mlflow.sklearn.save_model(model, "saved_model")
  File "c:\Keshav\Projects\Text-Classification-MLOPS\venv\Lib\site-packages\mlflow\sklearn

🏃 View run resilient-roo-700 at: https://dagshub.com/keshav-github-123/Text-Classification-MLOPS.mlflow/#/experiments/0/runs/e3b9edd361e7416b8a3dc473e1fc49d0
🧪 View experiment at: https://dagshub.com/keshav-github-123/Text-Classification-MLOPS.mlflow/#/experiments/0
